In [1]:
#!cp /content/drive/MyDrive/Colab/Data/'Готовые задачи'/Калининград/participants/train/train.csv ./ 
#!cp /content/drive/MyDrive/Colab/Data/'Готовые задачи'/Калининград/participants/test/test.csv ./ 

In [2]:
#Установка catboost
#!pip install catboost

In [3]:
import time
start_time = time.time()

In [4]:
#import необходимых модулей

import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [5]:
#Считывание данных в DataFrame 

train = pd.read_csv('train.csv', sep=';', index_col=None, dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str, 'PATIENT_ID_COUNT':int})
test = pd.read_csv('test.csv', sep=';', index_col=None, dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str})

### Исследование данных

In [6]:
train.head()

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00.0,Гурьевск,08.21,young,1
1,0,A00.0,Калининград,03.20,children,1
2,0,A00,Гусев,03.19,children,1
3,0,A00,Калининград,01.22,children,1
4,0,A00,Калининград,02.18,children,1


In [7]:
test.head()

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY
0,0,A00,Калининград,04.22,children
1,0,A00,Калининград,04.22,elderly
2,0,A00,Калининград,04.22,middleage
3,0,A00,Калининград,04.22,young
4,0,A01,Калининград,04.22,middleage


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212393 entries, 0 to 2212392
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   PATIENT_SEX       object
 1   MKB_CODE          object
 2   ADRES             object
 3   VISIT_MONTH_YEAR  object
 4   AGE_CATEGORY      object
 5   PATIENT_ID_COUNT  int32 
dtypes: int32(1), object(5)
memory usage: 92.8+ MB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39373 entries, 0 to 39372
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PATIENT_SEX       39373 non-null  object
 1   MKB_CODE          39373 non-null  object
 2   ADRES             39373 non-null  object
 3   VISIT_MONTH_YEAR  39373 non-null  object
 4   AGE_CATEGORY      39373 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [10]:
train.isna().sum()

PATIENT_SEX         0
MKB_CODE            0
ADRES               0
VISIT_MONTH_YEAR    0
AGE_CATEGORY        0
PATIENT_ID_COUNT    0
dtype: int64

In [11]:
train.describe()

,PATIENT_ID_COUNT
count,2.212393e+06
mean,5.847853e+00
std,6.042780e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,1.353200e+04


In [12]:
train['PATIENT_SEX'].value_counts()

0    1316709
1     895684
Name: PATIENT_SEX, dtype: int64

In [13]:
train['MKB_CODE'].value_counts()

J06.9    19100
I11.9    17282
Z00.0    15419
Z02.7    13458
K02.1    13132
         ...  
D76.1        1
Z44.3        1
D80.7        1
D82.9        1
Z96.2        1
Name: MKB_CODE, Length: 7644, dtype: int64

In [14]:
train['ADRES'].value_counts()

Калининград           617129
Гурьевск              116111
Светлый                85748
Пионерский             84932
Советск                78466
                       ...  
Ясная Поляна            2670
Малинники               2649
Гурьевский              2635
СТ Радуга               2588
СТ Железнодорожник      2517
Name: ADRES, Length: 118, dtype: int64

In [15]:
train['VISIT_MONTH_YEAR'].value_counts()

10.19    63145
04.19    61646
12.19    60759
10.18    60258
11.18    58999
03.19    58933
07.19    58811
11.19    58485
09.19    57688
02.19    57278
12.18    56813
03.18    56510
08.19    56411
04.18    56068
07.18    55876
05.19    55516
02.18    55369
06.19    55302
06.18    55048
08.18    55040
05.18    55023
09.18    53682
01.19    53661
01.18    50770
03.22    36298
10.20    35684
09.20    35458
04.21    34922
12.21    34720
06.21    34547
09.21    34471
03.20    34416
02.20    34376
03.21    34137
07.21    33806
11.21    33548
08.21    33153
07.20    33073
10.21    33057
05.21    32948
08.20    32434
01.20    30550
11.20    29442
02.21    29176
06.20    29166
12.20    28624
01.22    25743
02.22    24348
01.21    22959
04.20    22893
05.20    21353
Name: VISIT_MONTH_YEAR, dtype: int64

In [16]:
train['AGE_CATEGORY'].value_counts()

young           552363
elderly         514680
middleage       438426
children        430511
old             214493
centenarians     61920
Name: AGE_CATEGORY, dtype: int64

In [17]:
train['PATIENT_ID_COUNT'].value_counts()

1       1366778
2        318988
3        136250
4         77480
5         50492
         ...   
1241          1
4501          1
1166          1
1415          1
6733          1
Name: PATIENT_ID_COUNT, Length: 1688, dtype: int64

In [46]:
test.describe()

,PATIENT_ID_COUNT
count,39373.000000
mean,6.357021
std,42.801128
min,0.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,2999.000000


In [18]:
# Преобразование VISIT_MONTH_YEAR to_datetime

In [19]:
train['VISIT_MONTH_YEAR']=train['VISIT_MONTH_YEAR'].astype(str)
train['VISIT_MONTH_YEAR'] = '01.' +  train['VISIT_MONTH_YEAR']
train['VISIT_MONTH_YEAR'] = pd.to_datetime(train['VISIT_MONTH_YEAR'], dayfirst=True)

In [20]:
test1['VISIT_MONTH_YEAR']=test['VISIT_MONTH_YEAR'].astype(str)
test1['VISIT_MONTH_YEAR'] = '01.' +  test1['VISIT_MONTH_YEAR']
test1['VISIT_MONTH_YEAR'] = pd.to_datetime(test1['VISIT_MONTH_YEAR'], dayfirst=True)

In [21]:
#Отделение меток от данных

X = train[['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'VISIT_MONTH_YEAR', 'AGE_CATEGORY']]
y = train[['PATIENT_ID_COUNT']]

In [22]:
#Разделение на train/test для локального тестирования

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [23]:
#Создание объекта данных Pool, плюсы: возможность указать какие признаки являются категориальными

pool_train = Pool(X_train, y_train, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])
pool_test = Pool(X_test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])

In [24]:
#Объявление CatBoostRegressor и обучение

#model = CatBoostRegressor(task_type='GPU')
model = CatBoostRegressor(depth = 7, iterations = 1000)
model.fit(pool_train,verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
#Получение ответов модели на тестовой выборке в локальном тестировании 

y_pred = model.predict(pool_test)

In [26]:
#На локальном тестировании модель выдаёт такой результат

print("Значение метрики R2 на test: ", r2_score(y_test, y_pred))

Значение метрики R2 на test:  0.9255703980885073


In [28]:
#Формируем sample_solution. В обучении используется весь train, ответы получаем на test

pool_train_solution = Pool(X, y, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])
pool_test_solution = Pool(test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])

#model_solution = CatBoostRegressor(task_type='GPU')
#model_solution = CatBoostRegressor(depth = 7, iterations = 1000)
#model_solution.fit(pool_train_solution,verbose=False, plot=True)

In [29]:
#Получение ответов

y_pred_solution = model.predict(pool_test_solution)

In [30]:
y_pred_solution

array([0.43364092, 1.79245329, 1.45745083, ..., 0.7282593 , 1.28414883,
       1.35069108])

In [31]:
#Вот так они выглядят

y_pred_solution.astype(int)

array([0, 1, 1, ..., 0, 1, 1])

In [40]:
#Формируем sample_solution для отправки на платформу

test['PATIENT_ID_COUNT'] = y_pred_solution.astype(int)

In [41]:
test

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00,Калининград,04.22,children,0
1,0,A00,Калининград,04.22,elderly,1
2,0,A00,Калининград,04.22,middleage,1
3,0,A00,Калининград,04.22,young,1
4,0,A01,Калининград,04.22,middleage,2
...,...,...,...,...,...,...
39368,1,Z96.6,Балтийск,04.22,elderly,1
39369,1,Z96.6,Гусев,04.22,middleage,1
39370,1,Z96.7,Гусев,04.22,young,0
39371,1,Z98.8,Озерск,04.22,children,1


In [33]:
# Обнуление отрицательных значений PATIENT_ID_COUNT

In [43]:
#test[test['PATIENT_ID_COUNT'] < 0]

In [44]:
test.loc[(test['PATIENT_ID_COUNT'] < 0), 'PATIENT_ID_COUNT'] = 0 

In [45]:
#Сохраняем в csv файл
 
test.to_csv('sample_solution.csv', sep=';', index=None)

In [ ]:
#model_solution.plot_tree(tree_idx=0)

In [38]:
# CatBoost дает нам словарь со всеми параметрами модели. Мы можем вывести их, как словарь.
#or key,value in model_solution.get_all_params().items():
for key,value in model.get_all_params().items():
     print('{}, {}'.format(key,value))

nan_mode, Min
eval_metric, RMSE
combinations_ctr, ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1', 'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1']
iterations, 1000
sampling_frequency, PerTree
fold_permutation_block, 0
leaf_estimation_method, Newton
counter_calc_method, SkipTest
grow_policy, SymmetricTree
penalties_coefficient, 1
boosting_type, Plain
model_shrink_mode, Constant
feature_border_type, GreedyLogSum
ctr_leaf_count_limit, 18446744073709551615
bayesian_matrix_reg, 0.10000000149011612
one_hot_max_size, 2
force_unit_auto_pair_weights, False
l2_leaf_reg, 3
random_strength, 1
rsm, 1
boost_from_average, True
max_ctr_complexity, 4
model_size_reg, 0.5
simple_ctr, ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1', 'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1']
pool_metainfo_options, {'tags': {}}
s

In [ ]:
# 220827_11 R2 на test:  0.9255703980885073  Score = 0.851569  №24 depth = 7 iterations 1000 random_state=1  test_size=0.20 -to0 to_datetime

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))